In [8]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
import keras
import matplotlib.pyplot as plt  
import tqdm

@keras.saving.register_keras_serializable()
def diff(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true)
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    y_true = tf.math.floormod(y_true, 720)
    y_pred = tf.math.floormod(y_pred, 720)

    return (y_pred - y_true + 360) % 720 - 360

@keras.saving.register_keras_serializable()
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(diff(y_true, y_pred)))

@keras.saving.register_keras_serializable()
def custom_accuracy(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.abs(diff(y_true, y_pred)) <= 20, tf.float32))


def diff_for_one(y_true, y_pred):   
    y_true = tf.math.floormod(y_true, 720)
    y_pred = tf.math.floormod(y_pred, 720)
    return (y_pred - y_true + 360) % 720 - 360


def difffffffffffffff(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true)
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    y_true = tf.math.floormod(y_true, 720)
    y_pred = tf.math.floormod(y_pred, 720)

    return (y_pred - y_true + 360) % 720 - 360

def custom_accuracyyyyyyyyyyyy(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.abs(difffffffffffffff(y_true, y_pred)) <= 20, tf.float32))

In [9]:
model_path = 'models/final_model.keras'
model = load_model(model_path, custom_objects={'custom_loss': custom_loss, 'custom_accuracy': custom_accuracy})

In [3]:
folder_path = [
    'c:\\Users\\mymri\\repos\\clock_set/clock_01',      'c:\\Users\\mymri\\repos\\clock_set/clock_02_1',    'c:\\Users\\mymri\\repos\\clock_set/clock_02_2',
    'c:\\Users\\mymri\\repos\\clock_set/clock_03',      'c:\\Users\\mymri\\repos\\clock_set/clock_04',      'c:\\Users\\mymri\\repos\\clock_set/clock_05',
    'c:\\Users\\mymri\\repos\\clock_set/clock_06',      'c:\\Users\\mymri\\repos\\clock_set/clock_07',      'c:\\Users\\mymri\\repos\\clock_set/clock_08_1',
    'c:\\Users\\mymri\\repos\\clock_set/clock_08_3',    'c:\\Users\\mymri\\repos\\clock_set/clock_09_1',    'c:\\Users\\mymri\\repos\\clock_set/clock_09_2',
    'c:\\Users\\mymri\\repos\\clock_set/clock_09_2_1',  'c:\\Users\\mymri\\repos\\clock_set/clock_09_3_1',  'c:\\Users\\mymri\\repos\\clock_set/clock_09_3_2',
    'c:\\Users\\mymri\\repos\\clock_set/clock_09_3_3',  'c:\\Users\\mymri\\repos\\clock_set/clock_10',      'c:\\Users\\mymri\\repos\\clock_set/clock_11',
    'c:\\Users\\mymri\\repos\\clock_set/clock_12',      'c:\\Users\\mymri\\repos\\clock_set/clock_13',      'c:\\Users\\mymri\\repos\\clock_set/clock_14',
    'c:\\Users\\mymri\\repos\\clock_set/clock_15',      'c:\\Users\\mymri\\repos\\clock_set/clock_16',      'c:\\Users\\mymri\\repos\\clock_set/clock_17',
    'c:\\Users\\mymri\\repos\\clock_set/clock_18',      'c:\\Users\\mymri\\repos\\clock_set/clock_19',      'c:\\Users\\mymri\\repos\\clock_set/clock_20',
    'c:\\Users\\mymri\\repos\\clock_set/clock_21',      'c:\\Users\\mymri\\repos\\clock_set/clock_22',      'c:\\Users\\mymri\\repos\\clock_set/clock_23']

In [4]:
total = 0
n = 0
diff_per_folder = []

for name in folder_path:
    labels_path = os.path.join(name, 'dataset_info.csv')
    labels_df = pd.read_csv(labels_path, usecols=[0, 1], header=None, names=['filename', 'label'])
    difference = []

    for index, row in labels_df.iterrows():
        if index != 0:
            img_file = row['filename']
            true_label = row['label']
            true_label = float(true_label)

            img_path = os.path.join(name, img_file)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Ошибка при загрузке изображения: {img_path}")
                continue

            img = cv2.resize(img, (224, 224))  
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            img_array = img.astype('float32') / 255.0  
            img_array = np.expand_dims(img_array, axis=0)  

            predictions = model.predict(img_array)
            diff = diff_for_one(true_label, predictions[0][0])
            difference.append(diff)
            accur = tf.abs(diff) <= 20
            total += int(accur)
            n += 1
    diff_per_folder.append(difference)
    print(f"acc after {name} = {total/n} ")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118m

In [ ]:
for i in range(25, len(diff_per_folder)):
    x = np.abs(diff_per_folder[i])
    count1 = np.sum(np.abs(x) < 5)
    count2 = np.sum(np.abs(x) < 20)
    print(f"average diff = {sum(x)/len(diff_per_folder[i])}; acc(5) = {count1/len(x)}; acc(20) = {count2/len(x)}")

average diff = 32.69739447699653; acc(5) = 0.12544802867383512; acc(20) = 0.4551971326164875
average diff = 67.40611195991544; acc(5) = 0.07885304659498207; acc(20) = 0.24014336917562723
average diff = 54.55723397683777; acc(5) = 0.06711409395973154; acc(20) = 0.2550335570469799
average diff = 65.73114323554738; acc(5) = 0.04627249357326478; acc(20) = 0.2390745501285347
average diff = 26.06307009128338; acc(5) = 0.1267605633802817; acc(20) = 0.49061032863849763


In [25]:
total = 0
n = 0

for i in range(len(diff_per_folder)):
    total += sum(np.abs(diff_per_folder[i]))
    n += len(diff_per_folder[i])

print(f"average diff = {total/n}") 

average diff = 27.49313082777254


In [27]:
count_5 = 0
for i in range(len(diff_per_folder)):
    count_5 += np.sum(np.abs(diff_per_folder[i]) < 5)
print(count_5/n)

0.1707020811401733


In [28]:
count_20 = 0
for i in range(len(diff_per_folder)):
    count_20 += np.sum(np.abs(diff_per_folder[i]) < 20)
print(count_20/n)

0.5491942667422464


In [35]:
total = 0
n = 0
count_5 = 0
count_20 = 0
indexes = [1, 2, 6, 7, 19, 20, 24, 25, 26, 27, 28, 29]
for i in indexes:
    total += sum(np.abs(diff_per_folder[i]))
    n += len(diff_per_folder[i])
    count_5 += np.sum(np.abs(diff_per_folder[i]) < 5)
    count_20 += np.sum(np.abs(diff_per_folder[i]) < 20)

print(f"average diff = {total/n}") 
print(count_5/n)
print(count_20/n)

average diff = 39.33705489021154
0.10828603859250852
0.39023836549375707
